In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()

import os
from vexa_client import VexaClient


In [2]:
admin_client = VexaClient(
    base_url="https://gateway.dev.vexa.ai",
    admin_key="token"  # From docker-compose.yml
)

In [3]:
import random

In [4]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")



DEBUG: Making POST request to https://gateway.dev.vexa.ai/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '923306@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Fri, 11 Apr 2025 09:04:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '111', 'Connection': 'keep-alive'}
DEBUG: Response content: {"email":"923306@example.com","name":"test","image_url":null,"id":14,"created_at":"2025-04-11T09:04:58.462111"}...


In [5]:

user_id = new_user['id']

In [6]:
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']


DEBUG: Making POST request to https://gateway.dev.vexa.ai/admin/users/14/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Fri, 11 Apr 2025 09:04:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive'}
DEBUG: Response content: {"user_id":14,"id":14,"token":"OFgxKZH4fRTJsDjP3w8XChM8T0gGY06Fp763C8D0","created_at":"2025-04-11T09:04:58.502053"}...


In [7]:
user_api_key

'OFgxKZH4fRTJsDjP3w8XChM8T0gGY06Fp763C8D0'

In [8]:
client = VexaClient(
    base_url="https://gateway.dev.vexa.ai",
    api_key=user_api_key
)

In [9]:
meeting_url="https://meet.google.com/tvq-jkyf-qgt"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"

In [10]:
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="MyBot"
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to https://gateway.dev.vexa.ai/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'OFgxKZH4fRTJsDjP3w8XChM8T0gGY06Fp763C8D0'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'tvq-jkyf-qgt', 'bot_name': 'MyBot'}
DEBUG: Response status: 201
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Fri, 11 Apr 2025 09:04:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '393', 'Connection': 'keep-alive'}
DEBUG: Response content: {"id":14,"user_id":14,"platform":"google_meet","native_meeting_id":"tvq-jkyf-qgt","constructed_meeting_url":"https://meet.google.com/tvq-jkyf-qgt","status":"active","bot_container_id":"41e18761a04f60fdbc9a8e1db72c48f59aa5e78c484d8923ebebf53c3d96c55c","start_time":"2025-04-11T09:04:58.805492","end_time":null,"created_at":"2025-04-11T09:04:58.564986","updated_at":"2025-04-11T09:04:58.580485"}...


In [ ]:
meetings = client.get_meetings()


DEBUG: Making GET request to https://gateway.dev.vexa.ai/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'kZTgAosSfDdL72sW7ZibiSmYz9PvXxFosSNePlHQ'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Thu, 10 Apr 2025 15:36:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '408', 'Connection': 'keep-alive'}
DEBUG: Response content: {"meetings":[{"id":13,"user_id":13,"platform":"google_meet","native_meeting_id":"fvb-qhmn-gkc","constructed_meeting_url":"https://meet.google.com/fvb-qhmn-gkc","status":"active","bot_container_id":"a03a68799be943cda00eaee39228a303974a40f03ff5d3b234dbade61da8c207","start_time":"2025-04-10T15:36:50.207364","end_time":null,"created_at":"2025-04-10T15:36:49.986065","updated_at":"2025-04-10T15:36:49.992155"}]}...


In [12]:
meeting_id

13

In [304]:
import pandas as pd
transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
segments = transcript.get('segments', [])
df = pd.DataFrame(transcript['segments']).sort_values(['created_at','start']).drop_duplicates('start',keep='last')
df


DEBUG: Making GET request to https://gateway.dev.vexa.ai/transcripts/google_meet/tvq-jkyf-qgt
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'OFgxKZH4fRTJsDjP3w8XChM8T0gGY06Fp763C8D0'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Fri, 11 Apr 2025 09:11:00 GMT', 'Content-Type': 'application/json', 'Content-Length': '13525', 'Connection': 'keep-alive'}
DEBUG: Response content: {"id":14,"platform":"google_meet","native_meeting_id":"tvq-jkyf-qgt","constructed_meeting_url":"https://meet.google.com/tvq-jkyf-qgt","status":"active","start_time":"2025-04-11T09:04:58.805492","end_time":null,"segments":[{"start":0.0,"end":7.32,"text":" работающей системы, как она будет определять русский язык, как это будет сейчас работать, но сейчас, смотрим, все нормально.","language":null,"created_at":"2025-04-11T09:05:24.642671","speaker":null},{"start":0.0,"end":7.709,"text":" Работающая ...


,start,end,text,language,created_at,speaker
5,0.000,5.000,"Работающая система, как она будет определять ...",None,2025-04-11T09:05:25.837974,None
7,5.000,8.000,"Но сейчас смотрим, все нормально, пора получше.",None,2025-04-11T09:05:25.838349,None
9,10.000,10.774,Ну вот,None,2025-04-11T09:05:27.946580,None
8,8.000,15.000,"Ну вот, было лучше. Вот всё, нам всё одну шту...",None,2025-04-11T09:05:47.075073,None
10,15.000,20.000,Гораздо лучше всё пошло.,None,2025-04-11T09:05:47.075455,None
12,15.270,33.270,"И теперь я перейду к английскому, и я буду го...",None,2025-04-11T09:05:50.937534,None
14,35.585,40.665,still it's transcribed into russian,None,2025-04-11T09:06:03.529225,None
21,43.585,47.145,still it's transcribed into russian,None,2025-04-11T09:06:04.545670,None
19,41.145,43.145,...,None,2025-04-11T09:06:08.351390,None
20,43.145,47.705,...,None,2025-04-11T09:06:08.351809,None


In [ ]:
df['text'].tolist()

[" Yeah, so tell me about about yourself. What's that that you're doing?",
 ' Is it about...',
 ' Yeah',
 " There's a bit of a risk.",
 ' Okay?',
 " Yeah, I've been switching another, the proprietary version now, the other, the other.",
 ' the other',
 ' Thank you very much.',
 " You're most welcome.",
 " I'm just curious what types of model I've been using for that, I suppose it's all because",
 " I suppose it's a little bit...",
 " about yourself? What's that you're doing? Are you about, you said you're about machine learning?",
 " I suppose it's a lot of kind of boosting trees, yes, this is like a pretty large language",
 ' Is it about STG and TGS things, have you played around this stuff?',
 ' model. Yeah, so mostly table type of table data, right?',
 ' So mostly table type of table data, right?',
 ' See, okay.',
 ' Thank you very much.',
 ' Yes, yes.',
 ' Yeah.',
 ' Thanks.',
 ' Thank you.',
 " Yeah, I've been switching another the proprietary policy now the other",
 " Okay. Okay.

In [25]:
platform = 'google_meet'

In [ ]:

    
def get_transcript_segments(native_meeting_id, platform):
    
    
    meeting_id = conn.execute(
        select([metadata.tables['meetings'].c.id]).where(
            and_(
                metadata.tables['meetings'].c.native_meeting_id == native_meeting_id,
                metadata.tables['meetings'].c.platform == platform
            )
        )
    ).scalar()
    
    if not meeting_id:
        conn.close()
        return []
    
    result = conn.execute(
        select([metadata.tables['transcript_segments']]).where(
            metadata.tables['transcript_segments'].c.meeting_id == meeting_id
        ).order_by(metadata.tables['transcript_segments'].c.start_time)
    ).fetchall()
    
    conn.close()
    return result

if __name__ == "__main__":
    print(len(get_transcript_segments("nav-yijy-spp", "google_meet"))) 

NameError: name 'conn' is not defined

In [ ]:
"""Vexa API Client - Minimal Usage Snippets"""
from vexa_client import VexaClient

# --- Client Initialization ---
# User client
client = VexaClient(
    base_url="http://localhost:8056",
    api_key="your-user-api-key"
)

# Admin client 
admin_client = VexaClient(
    base_url="http://localhost:8056",
    admin_key="supersecretadmintoken"  # From docker-compose.yml
)

# --- Admin Operations ---
# Create user
new_user = admin_client.create_user(email="user@example.com", name="New User")
user_id = new_user['id']